<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/sentencepiece_embeding_for_llama_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece

In [4]:
import pandas as pd
ds = pd.read_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')
ds.rename(columns = {'Unnamed: 0':'ids'}, inplace = True)
ds

,ids,token
0,0,<unk>
1,1,<s>
2,2,</s>
3,3,<0x00>
4,4,<0x01>
...,...,...
31995,31995,黃
31996,31996,왕
31997,31997,收
31998,31998,弘


In [ ]:
alefba={'ا','ب','پ','ت','ث','ج','چ','ح','خ','د','ذ','ر','ز','ژ','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ک','گ','ل','م','ن','و','ه','ی','ء','آ','اً','هٔ','ة'}

dffa = pd.DataFrame()
for i in alefba:
  s = ds[ds['token']==i]
  dffa = dffa.append(s)

In [6]:
dffa = dffa.append(ds.iloc[6756])

<ipython-input-6-872a4eb9c750>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dffa = dffa.append(ds.iloc[6756])


In [7]:
dffa

,ids,token
30171,30171,و
30585,30585,پ
30421,30421,گ
30202,30202,ی
30240,30240,ح
30992,30992,ء
30241,30241,ف
30726,30726,آ
30270,30270,ج
30138,30138,ل


In [8]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('/content/drive/MyDrive/sentencepiesese/vocab_16000.model')

True

In [9]:
vocab_16000 = pd.read_csv('/content/drive/MyDrive/sentencepiesese/vocab_16000.vocab', header=None,delimiter='\t')
vocab_16000.rename(columns = {0:'token',1:'score'}, inplace = True)
vocab_16000

,token,score
0,<unk>,0.00000
1,<s>,0.00000
2,</s>,0.00000
3,▁و,-3.35953
4,▁در,-3.38457
...,...,...
15995,؛,-13.63230
15996,],-14.45220
15997,»,-16.73590
15998,(,-16.73600


In [10]:
def tokne2tokenize(token):
    listtoken = []
    unkn = []
    done = True
    for l in token:
        ids = dffa[dffa['token']==l]['ids']
        #print(token,"  l=>",l,"  ids:",ids)
        #print('ids',ids)
        #print('ids',type(ids))
        if len(ids) == 1:
            listtoken.append(ids.item())
        else:
            #print(token,"  l=>",l,"  ids:",ids)
            listtoken.append('unkn')
            done = False
            break
            
            #unkn.append(ids)
    return listtoken , done

#sample['tokenizer'] = sample['token'].apply(tokne2tokenize)
vocab_16000[['tokenizer', 'done']] = vocab_16000['token'].apply(tokne2tokenize).apply(pd.Series)
vocab_16000

,token,score,tokenizer,done
0,<unk>,0.00000,[unkn],False
1,<s>,0.00000,[unkn],False
2,</s>,0.00000,[unkn],False
3,▁و,-3.35953,"[6756, 30171]",True
4,▁در,-3.38457,"[6756, 30172, 30156]",True
...,...,...,...,...
15995,؛,-13.63230,[unkn],False
15996,],-14.45220,[unkn],False
15997,»,-16.73590,[unkn],False
15998,(,-16.73600,[unkn],False


In [21]:
final_voacb = vocab_16000[vocab_16000['done']==True]
final_voacb

,token,score,tokenizer,done
3,▁و,-3.35953,"[6756, 30171]",True
4,▁در,-3.38457,"[6756, 30172, 30156]",True
7,▁به,-3.68497,"[6756, 30177, 30204]",True
8,▁از,-3.91135,"[6756, 30112, 30295]",True
9,ی,-3.98119,[30202],True
...,...,...,...,...
15989,▁بدهکار,-12.78280,"[6756, 30177, 30172, 30204, 30343, 30112, 30156]",True
15990,ساندرو,-12.78320,"[30198, 30112, 30162, 30172, 30156, 30171]",True
15991,▁عطایی,-12.78330,"[6756, 30218, 30367, 30112, 30202, 30202]",True
15992,ایزوله,-12.78380,"[30112, 30202, 30295, 30171, 30138, 30204]",True


In [12]:
import torch
emmbding = torch.load('/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [22]:
#final_voacb['index'] = final_voacb.index
final_voacb = final_voacb.reset_index()
final_voacb

,index,token,score,tokenizer,done
0,3,▁و,-3.35953,"[6756, 30171]",True
1,4,▁در,-3.38457,"[6756, 30172, 30156]",True
2,7,▁به,-3.68497,"[6756, 30177, 30204]",True
3,8,▁از,-3.91135,"[6756, 30112, 30295]",True
4,9,ی,-3.98119,[30202],True
...,...,...,...,...,...
13709,15989,▁بدهکار,-12.78280,"[6756, 30177, 30172, 30204, 30343, 30112, 30156]",True
13710,15990,ساندرو,-12.78320,"[30198, 30112, 30162, 30172, 30156, 30171]",True
13711,15991,▁عطایی,-12.78330,"[6756, 30218, 30367, 30112, 30202, 30202]",True
13712,15992,ایزوله,-12.78380,"[30112, 30202, 30295, 30171, 30138, 30204]",True


In [34]:
def calc_embeding(tokenizer):
    sum_embd = 0
    for l in tokenizer:
        embdbase = emmbding[l].detach().numpy()
        sum_embd = (sum_embd + embdbase)/2
    return sum_embd#/len(tokenizer)

#sample['tokenizer'] = sample['token'].apply(tokne2tokenize)
final_voacb['embeding'] = final_voacb['tokenizer'].apply(calc_embeding)
final_voacb

,index,token,score,tokenizer,done,embeding,similarity
0,3,▁و,-3.35953,"[6756, 30171]",True,"[0.00804, 0.000759, 0.007378, 0.002293, 0.0219...",0.981934
1,4,▁در,-3.38457,"[6756, 30172, 30156]",True,"[0.003014, -0.027, -0.001198, -0.0123, -0.0183...",0.168213
2,7,▁به,-3.68497,"[6756, 30177, 30204]",True,"[-0.00138, -0.02017, 0.001684, 0.01631, 0.0046...",0.066711
3,8,▁از,-3.91135,"[6756, 30112, 30295]",True,"[-0.010056, -0.02255, 0.0344, 0.039, -0.01857,...",0.060699
4,9,ی,-3.98119,[30202],True,"[-0.003796, 0.0001017, 0.009995, -0.006817, -0...",0.085083
...,...,...,...,...,...,...,...
13709,15989,▁بدهکار,-12.78280,"[6756, 30177, 30172, 30204, 30343, 30112, 30156]",True,"[-0.00628, -0.02394, -0.00371, 0.00595, -0.015...",0.171509
13710,15990,ساندرو,-12.78320,"[30198, 30112, 30162, 30172, 30156, 30171]",True,"[0.01393, -0.0232, -0.003897, -0.00667, 0.0099...",1.027344
13711,15991,▁عطایی,-12.78330,"[6756, 30218, 30367, 30112, 30202, 30202]",True,"[-0.00908, 0.000699, 0.01743, -0.001873, -0.02...",0.122925
13712,15992,ایزوله,-12.78380,"[30112, 30202, 30295, 30171, 30138, 30204]",True,"[-0.0001285, -0.03375, 0.005196, 0.010345, 0.0...",0.261230


In [40]:
import numpy as np
px = final_voacb
#text_indexbase = tokenizer.encode('n')
##print(text_indexbase)
text_indexbase = 6675
embdbase = final_voacb.iloc[text_indexbase]['embeding']
print(final_voacb.iloc[text_indexbase]['token'])
print(embdbase.shape)
max=0

print()

def calc_similarity(embeding):
    embd = embeding
    smltry = (np.dot(embd,embdbase)).item()
    return smltry

px['similarity'] = px['embeding'].apply(calc_similarity)

px.sort_values(['similarity'], ascending=False).head(20)

غل
(4096,)



,index,token,score,tokenizer,done,embeding,similarity
2566,2860,▁الملل,-10.06670,"[6756, 30112, 30138, 30159, 30138, 30138]",True,"[-0.007645, -0.03958, 0.0115, -0.01364, 0.0007...",0.838379
11417,13176,▁مجلل,-12.22270,"[6756, 30159, 30270, 30138, 30138]",True,"[-0.005863, -0.0392, 0.01192, -0.01435, 0.0001...",0.781738
1393,1559,▁ملل,-9.37502,"[6756, 30159, 30138, 30138]",True,"[-0.006714, -0.03525, 0.012146, -0.01346, 0.00...",0.777832
5617,6401,▁علل,-11.06180,"[6756, 30218, 30138, 30138]",True,"[-0.00931, -0.03665, 0.01447, -0.012405, 0.003...",0.770508
9142,10461,▁لل,-11.77760,"[6756, 30138, 30138]",True,"[-0.00753, -0.03476, 0.01825, -0.01315, 0.0051...",0.761230
6876,7855,▁الل,-11.34430,"[6756, 30112, 30138, 30138]",True,"[-0.010284, -0.03647, 0.01672, -0.00751, 0.004...",0.754395
5989,6836,▁فلفل,-11.15040,"[6756, 30241, 30138, 30241, 30138]",True,"[-0.0016, -0.04565, 0.00944, -0.00853, -0.0094...",0.688965
10546,12116,▁بلبل,-12.04620,"[6756, 30177, 30138, 30177, 30138]",True,"[-0.00627, -0.03885, 0.01047, -0.00823, 0.0102...",0.684570
7649,8753,▁مسلسل,-11.50570,"[6756, 30159, 30198, 30138, 30198, 30138]",True,"[-0.000765, -0.0373, 0.00702, -0.01776, -0.004...",0.672852
3021,3373,▁محلول,-10.26510,"[6756, 30159, 30240, 30138, 30171, 30138]",True,"[-0.0003662, -0.0345, 0.010086, -0.01325, 0.01...",0.671387
